<a href="https://colab.research.google.com/github/withswim/data-engineering-batch5/blob/master/%5B_Sooyoung%5D_Airflow_1%EC%A3%BC%EC%B0%A8_%EC%88%99%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. INSERT문 완성**

In [ ]:
import requests

def extract(url):
  f = requests.get(url)
  return (f.text)

In [ ]:
import csv

def transform(data):
  reader = csv.reader(data.strip().split("\n"))
  return reader

In [ ]:
import psycopg2

# Redshift connection 함수
def get_Redshift_connection():
    host = "learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com"
    redshift_user = "ID"
    redshift_pass = "PW"
    port = 5439
    dbname = "dev"
    conn = psycopg2.connect("dbname={dbname} user={user} host={host} password={password} port={port}".format(
        dbname=dbname,
        user=redshift_user,
        password=redshift_pass,
        host=host,
        port=port
    ))
    conn.set_session(autocommit=True)
    return conn.cursor()

"""
CREATE TABLE raw_data.name_gender (
  name varchar(64),
  gender varchar(16)
)
"""

def load(csv_reader):

    try:
        # create a conncection
        cur = get_Redshift_connection()
      
    except Exception as err:
        print("Exception occured while creating a connection", err)

    else:
        sql = """CREATE TALBE IF NOT EXISTS name_gender (
            name varchar(64),
            gender varchar(16)
            );
            DELETE FROM name_gender;"""
        cur.execute(sql)

        for name, gender in csv_reader:
            sql = """INSERT INTO RAW_DATA.NAME_GENDER (NAME, GENDER) VALUES (%s, %s);"""
            cur.execute(sql)

        cur.close()

   

SyntaxError: ignored

**Extraxt부터 함수를 하나씩 실행**

In [ ]:
link = "https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv"

data = extract(link)
csv_reader = transform(data)
load(csv_reader)

Exception occured while creating a connection could not translate host name "learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com" to address: Name or service not known



# **2. OpenWeathermap API: 서울 7일 평균/최소/최대 온도 읽어오기** 


확인 포인트:
1. api에서 제공하는 온도는 **Kalvin Degrees**인 것으로 보여 **celsius**로 변환하는 함수를 만들었음
2. api에서 받아온, current.dt는 UTC 날짜이고 daily.temp.dt는 해당지역(서울) 기준 날짜로 만들어진 것으로 보임. 시차에 따라 첫번째 데이터가 동일날짜로 보일 수 있음.
   향후 7일간의 날짜는 api에서 제공하는 총 8개의 데이터를 모두 보여주었음.

In [ ]:
import requests
from datetime import date
import json

url = 'https://api.openweathermap.org/data/2.5/onecall?lat=35.9078&lon=127.7669&appid=56b50656581980953614f5a047b719ab'
r = requests.get(url)

data = r.json()

def to_celsius(kevin_temp):
  return kevin_temp - 273.15

forecast = data["daily"]

print("Today is %s" %date.fromtimestamp(data["current"]["dt"]).strftime('%Y-%m-%d'))
print("Weather forecast for the next 7 days")

for i in range(8):
  print("%s  day:%3d  min:%3d  max:%3d" % ( date.fromtimestamp(forecast[i]["dt"]).strftime('%Y-%m-%d'), round(to_celsius(forecast[i]["temp"]["day"])),round(to_celsius(forecast[i]["temp"]["min"])),round(to_celsius(forecast[i]["temp"]["max"])) )   )


Today is 2021-10-17
Weather forecast for the next 7 days
2021-10-17  day:  8  min: -3  max:  8
2021-10-18  day: 12  min:  0  max: 12
2021-10-19  day: 11  min:  1  max: 11
2021-10-20  day: 10  min:  0  max: 11
2021-10-21  day: 11  min:  2  max: 11
2021-10-22  day: 13  min:  4  max: 13
2021-10-23  day: 14  min:  4  max: 15
2021-10-24  day: 16  min:  6  max: 16
